In [1]:
%cd src

D:\study\Capstone\bertAeslc\src


!python train.py -task abs \
                 -mode validate \
                 -batch_size 3000 \
                 -test_batch_size 500 \
                 -bert_data_path ../bert_data \
                 -log_file ../logs/val_abs_bert_aeslc \
                 -model_path ../models/abs/ \
                 -sep_optim true \
                 -use_interval true \
                 -visible_gpus 0 \
                 -max_pos 512 \
                 -max_length 200 \
                 -alpha 0.95 \
                 -min_length 50 \
                 -result_path ../logs/abs_bert_aeslc

In [2]:
import argparse
import os
from others.logging import init_logger
from train_abstractive import validate_abs, train_abs, baseline, test_abs, test_text_abs
from train_extractive import train_ext, validate_ext, test_ext

model_flags = ['hidden_size', 'ff_size', 'heads', 'emb_size', 'enc_layers', 'enc_hidden_size', 'enc_ff_size',
               'dec_layers', 'dec_hidden_size', 'dec_ff_size', 'encoder', 'ff_actv', 'use_interval']

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("-task", default='ext', type=str, choices=['ext', 'abs'])
parser.add_argument("-encoder", default='bert', type=str, choices=['bert', 'baseline'])
parser.add_argument("-mode", default='train', type=str, choices=['train', 'validate', 'test'])
parser.add_argument("-bert_data_path", default='../bert_data_new/cnndm')
parser.add_argument("-model_path", default='../models/')
parser.add_argument("-result_path", default='../results/cnndm')
parser.add_argument("-temp_dir", default='../temp')

parser.add_argument("-batch_size", default=140, type=int)
parser.add_argument("-test_batch_size", default=200, type=int)

parser.add_argument("-max_pos", default=512, type=int)
parser.add_argument("-use_interval", type=bool, nargs='?',const=True,default=True)
parser.add_argument("-large", type=bool, nargs='?',const=True,default=False)
parser.add_argument("-load_from_extractive", default='', type=str)

parser.add_argument("-sep_optim", type=bool, nargs='?',const=True,default=False)
parser.add_argument("-lr_bert", default=2e-3, type=float)
parser.add_argument("-lr_dec", default=2e-3, type=float)
parser.add_argument("-use_bert_emb", type=bool, nargs='?',const=True,default=False)

parser.add_argument("-share_emb", type=bool, nargs='?', const=True, default=False)
parser.add_argument("-finetune_bert", type=bool, nargs='?', const=True, default=True)
parser.add_argument("-dec_dropout", default=0.2, type=float)
parser.add_argument("-dec_layers", default=6, type=int)
parser.add_argument("-dec_hidden_size", default=768, type=int)
parser.add_argument("-dec_heads", default=8, type=int)
parser.add_argument("-dec_ff_size", default=2048, type=int)
parser.add_argument("-enc_hidden_size", default=512, type=int)
parser.add_argument("-enc_ff_size", default=512, type=int)
parser.add_argument("-enc_dropout", default=0.2, type=float)
parser.add_argument("-enc_layers", default=6, type=int)

# params for EXT
parser.add_argument("-ext_dropout", default=0.2, type=float)
parser.add_argument("-ext_layers", default=2, type=int)
parser.add_argument("-ext_hidden_size", default=768, type=int)
parser.add_argument("-ext_heads", default=8, type=int)
parser.add_argument("-ext_ff_size", default=2048, type=int)

parser.add_argument("-label_smoothing", default=0.1, type=float)
parser.add_argument("-generator_shard_size", default=32, type=int)
parser.add_argument("-alpha",  default=0.6, type=float)
parser.add_argument("-beam_size", default=5, type=int)
parser.add_argument("-min_length", default=15, type=int)
parser.add_argument("-max_length", default=150, type=int)
parser.add_argument("-max_tgt_len", default=140, type=int)



parser.add_argument("-param_init", default=0, type=float)
parser.add_argument("-param_init_glorot", type=bool, nargs='?',const=True,default=True)
parser.add_argument("-optim", default='adam', type=str)
parser.add_argument("-lr", default=1, type=float)
parser.add_argument("-beta1", default= 0.9, type=float)
parser.add_argument("-beta2", default=0.999, type=float)
parser.add_argument("-warmup_steps", default=8000, type=int)
parser.add_argument("-warmup_steps_bert", default=8000, type=int)
parser.add_argument("-warmup_steps_dec", default=8000, type=int)
parser.add_argument("-max_grad_norm", default=0, type=float)

parser.add_argument("-save_checkpoint_steps", default=5, type=int)
parser.add_argument("-accum_count", default=1, type=int)
parser.add_argument("-report_every", default=1, type=int)
parser.add_argument("-train_steps", default=1000, type=int)
parser.add_argument("-recall_eval", type=bool, nargs='?',const=True,default=False)


parser.add_argument('-visible_gpus', default='-1', type=str)
parser.add_argument('-gpu_ranks', default='0', type=str)
parser.add_argument('-log_file', default='../logs/cnndm.log')
parser.add_argument('-seed', default=666, type=int)

parser.add_argument("-test_all", type=bool, nargs='?',const=True,default=True)
parser.add_argument("-test_from", default='')
parser.add_argument("-test_start_from", default=-1, type=int)

parser.add_argument("-train_from", default='')
parser.add_argument("-report_rouge", type=bool, nargs='?',const=True,default=True)
parser.add_argument("-block_trigram", type=bool, nargs='?', const=True, default=True)

args = parser.parse_args()

In [7]:
args.task = "abs" 
args.mode = "validate" 
args.batch_size = 3000 
args.test_batch_size = 500 
args.bert_data_path = "../bert_data/bert"
args.log_file = "../logs/val_abs_bert_aeslc"
args.model_path = "../models/abs/"
args.sep_optim = True 
args.use_interval = True 
args.visible_gpus = "-1" 
args.max_pos = 512 
args.max_length = 200 
args.alpha = 0.95 
args.min_length = 50 
args.result_path = "../logs/abs_bert_aeslc"

args.gpu_ranks = [int(i) for i in range(len(args.visible_gpus.split(',')))]
args.world_size = len(args.gpu_ranks)
os.environ["CUDA_VISIBLE_DEVICES"] = args.visible_gpus

init_logger(args.log_file)
device = "cpu" if args.visible_gpus == '-1' else "cuda"
device_id = 0 if device == "cuda" else -1

In [9]:
cp = "../models/abs/model_step_4000.pt"
step = int(cp.split('.')[-2].split('_')[-1])
test_abs(args, device_id, cp, step)

[2019-10-06 00:54:12,285 INFO] Loading checkpoint from ../models/abs/model_step_4000.pt


Namespace(accum_count=1, alpha=0.95, batch_size=3000, beam_size=5, bert_data_path='../bert_data/bert', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_aeslc', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='validate', model_path='../models/abs/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../logs/abs_bert_aeslc', save_checkpoint_steps=5, seed=666, sep_optim=True, share_emb=False, task='abs', temp_dir='../temp', test_all=True, test_batch_size=500, test_

[2019-10-06 00:54:27,333 INFO] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at ../temp\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
[2019-10-06 00:54:27,337 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2019-10-06 00:54:28,426 INFO] loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin from cache at ../temp\aa1ef1

1906
1906


2019-10-06 04:00:40,361 [MainThread  ] [INFO ]  Writing summaries.
[2019-10-06 04:00:40,361 INFO] Writing summaries.
2019-10-06 04:00:40,364 [MainThread  ] [INFO ]  Processing summaries. Saving system files to ../temp\tmpi163kow4\system and model files to ../temp\tmpi163kow4\model.
[2019-10-06 04:00:40,364 INFO] Processing summaries. Saving system files to ../temp\tmpi163kow4\system and model files to ../temp\tmpi163kow4\model.
2019-10-06 04:00:40,365 [MainThread  ] [INFO ]  Processing files in ../temp\rouge-tmp-2019-10-06-04-00-38/candidate/.
[2019-10-06 04:00:40,365 INFO] Processing files in ../temp\rouge-tmp-2019-10-06-04-00-38/candidate/.
2019-10-06 04:00:41,396 [MainThread  ] [INFO ]  Saved processed files to ../temp\tmpi163kow4\system.
[2019-10-06 04:00:41,396 INFO] Saved processed files to ../temp\tmpi163kow4\system.
2019-10-06 04:00:41,398 [MainThread  ] [INFO ]  Processing files in ../temp\rouge-tmp-2019-10-06-04-00-38/reference/.
[2019-10-06 04:00:41,398 INFO] Processing file

OSError: [WinError 193] %1 不是有效的 Win32 应用程序。